In [4]:
# libraries to use
import requests
import pandas as pd
import json
import io
import itertools
import datetime
import numpy as np
import math

In [ ]:
# get game logs for PlayerID from NBA.com (MeasureType=Base,Advanced,Misc,Scoring,Usage)
def get_game_logs_df(PlayerID, MeasureType, Season, PerMode):
    glogs_url = 'https://stats.nba.com/stats/playergamelogs?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType='+MeasureType+'&Month=0&OppTeamID=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode='+PerMode+'&Period=0&PlayerID='+str(PlayerID)+'&PlusMinus=N&Rank=N&Season='+Season+'&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&TeamID=&VsConference=&VsDivision='
    glogs_response = requests.get(url=glogs_url, headers=headers).json()
    glogs_headers = glogs_response['resultSets'][0]['headers']
    glogs_data = glogs_response['resultSets'][0]['rowSet']
    glogs_df = pd.DataFrame(data=glogs_data, columns=glogs_headers)
    return glogs_df

In [ ]:
def get_games_past_week(this_game, plyr_box_stats_df):
    game_date = this_game['GAME_DATE_OBJ']
    # for a given game_date find the week period before
    time_delta_yesterday = datetime.timedelta(days = 1)
    time_delta_week_ago = datetime.timedelta(days = 7)
    date_yesterday = game_date - time_delta_yesterday
    date_week_ago = game_date - time_delta_week_ago    
    # find games that lie between date_yesterday and date_week_ago
    game_dates_past_week = plyr_box_stats_df.loc[(plyr_box_stats_df['GAME_DATE_OBJ'] <= date_yesterday) & (plyr_box_stats_df['GAME_DATE_OBJ'] >= date_week_ago)]
    return game_dates_past_week

In [ ]:
# to-do: support for multiple teams for the case player is traded mid-season
# (exmpale: James Harden 2021-22 traded BRK -> PHI)

def calc_injury_stats(plyr_box_stats_df, team, last_injury_date, num_career_injuries):
    player_games = plyr_box_stats_df.loc[plyr_box_stats_df['TEAM_ABBREVIATION']==team]
    # reverse player_games since team schedule first -> last
    player_games.sort_values(by='GAME_DATE_OBJ', ascending=True, inplace = True)
    player_dates = player_games['GAME_DATE_OBJ']
    # to-do: use for later (multi-team support)
    max_date = max(player_dates)
    min_date = min(player_dates)
    # global: team_dates_dict
    team_schedule = team_dates_dict.get(team)
    team_dates = team_schedule['dateGame_OBJ']
    # to-do: use for later (multi-team support)
    team_max_date = max(team_dates)
    team_min_date = min(team_dates)
    # get list of games in team schedule player wasn't in
    missed_dates = np.setdiff1d(team_dates,player_dates, assume_unique=False)
    played_schedule = []
    for date in team_dates:
        if date in missed_dates:
            played_schedule.append(False)
        else:
            played_schedule.append(True)
    
    # create temp table of team schedule with player in game
    temp_df = pd.DataFrame({'GAME_DATE_OBJ':team_dates,
                           'PLAYED_GAME':played_schedule,})
    temp_df.reset_index(drop=True, inplace=True)

    # create column flagging player out next game
    out_next_game = []
    for index, current_date in temp_df.iterrows():
        # if at last player's game of regular season just put true (data doesn't extend to playoffs)
        if index == len(temp_df) - 1:
            out_next_game.append(0)
        else:
            next_date = temp_df.iloc[index + 1]
            if next_date['PLAYED_GAME']:
                out_next_game.append(0)
            else:
                out_next_game.append(1)
    temp_df.loc[:,'OUT_NEXT_GAME']  = out_next_game

    # get OUT_NEXT_GAME subset based on player game logs
    player_injury_data = temp_df.loc[temp_df['GAME_DATE_OBJ'].isin(player_dates)]

    # calculate days last injury and rolling career injuries during season
    days_last_injury = []
    date_last_injury = last_injury_date
    career_injuries = []
    career_injuries_count = num_career_injuries
    for idx, row in player_injury_data.iterrows():
        if row['OUT_NEXT_GAME'] == 1:
            career_injuries_count = career_injuries_count + 1
            date_last_injury = row['GAME_DATE_OBJ']
    
        time_delta = row['GAME_DATE_OBJ'] - date_last_injury
        days_last_injury.append(time_delta.days)
        career_injuries.append(career_injuries_count)

    player_injury_data.loc[:,'CAREER_INJURIES'] = career_injuries
    player_injury_data.loc[:,'DAYS_LAST_INJURY'] = days_last_injury
        
    # reverse the injury list  because player game logs/plyr_box_stats_df last -> first
    player_injury_data = player_injury_data.iloc[::-1]
    player_injury_data.reset_index(drop=True, inplace=True)
    
    plyr_box_stats_df.loc[:,'OUT_NEXT_GAME'] = player_injury_data['OUT_NEXT_GAME']
    plyr_box_stats_df.loc[:,'CAREER_INJURIES'] = player_injury_data['CAREER_INJURIES']
    plyr_box_stats_df.loc[:,'DAYS_LAST_INJURY'] = player_injury_data['DAYS_LAST_INJURY']
    
    return plyr_box_stats_df


In [ ]:
def get_full_game_logs_df(PlayerID, MeasureType, Season, PerMode):
    # get player game logs for PlayerID from NBA.com for all measure types (MeasureType=Base,Advanced,Misc,Scoring,Usage)
    # PlayerID = 1629029 # Luka Doncic
    # Season='2021-22'
    # PerMode='Totals'
    plyr_base_df = get_game_logs_df(PlayerID,'Base',Season, PerMode)
    plyr_advanced_df = get_game_logs_df(PlayerID,'Advanced',Season, PerMode)
    plyr_misc_df = get_game_logs_df(PlayerID,'Misc',Season, PerMode)
    plyr_scor_df = get_game_logs_df(PlayerID,'Scoring',Season, PerMode)
    plyr_usage_df = get_game_logs_df(PlayerID,'Usage',Season, PerMode)

    # get subset of attributes from each game log measure
    plyr_base_df_subset = plyr_base_df[plyr_base_headers_subset]
    plyr_advanced_df_subset = plyr_advanced_df[plyr_advanced_headers_subset]
    plyr_misc_df_subset = plyr_misc_df[plyr_misc_headers_subset]
    plyr_scor_df_subset = plyr_scor_df[plyr_scor_headers_subset]

    # put all game log subsets for player into one dataframe
    plyr_box_stats_df = pd.concat([plyr_base_df_subset, 
                                   plyr_advanced_df_subset, 
                                   plyr_misc_df_subset, 
                                   plyr_scor_df_subset], axis=1)

    num_box_stats = plyr_box_stats_df.shape[0]

    # Add Bio data to game log dataframe: Age, Height (in.), Weight (lbs.)
    # global: plyrs_bio_df
    player_bio = plyrs_bio_df.loc[plyrs_bio_df['PLAYER_ID'] == PlayerID]

    # cannot find player birthday info on NBA.com
    # subtract age by one when going back single NBA season; this is the work-around for now
    # As of 3/2/2023: for 2021-22 season, substract 1 from age at NBA.com
    player_age = [int(player_bio.iloc[0]['AGE'])-1] * num_box_stats
    plyr_box_stats_df['AGE'] = player_age

    player_height = [player_bio.iloc[0]['PLAYER_HEIGHT_INCHES']] * num_box_stats
    plyr_box_stats_df['PLAYER_HEIGHT_INCHES'] = player_height

    player_weight = [player_bio.iloc[0]['PLAYER_WEIGHT']] * num_box_stats
    plyr_box_stats_df['PLAYER_WEIGHT'] = player_weight

    # calculate field goals made mid-range and in the paint
    fg_attr = ['FGM','FGA','FG3M','FG3A','FTM','FTA','PTS','PTS_PAINT','PCT_PTS_2PT_MR']

    fgm_paint_list = []
    fgm_mr_list = []
    for x in range(0, num_box_stats):
        game_stats = plyr_box_stats_df.iloc[x][fg_attr]
        fgm_paint = game_stats['PTS_PAINT']/2
        fgm_2p = game_stats['FGM'] - game_stats['FG3M']
        fgm_mr = fgm_2p - fgm_paint
        fgm_paint_list.append(int(fgm_paint))
        fgm_mr_list.append(int(fgm_mr))

    # note: cannot find 2PT Mid-range FG attempts in player box scores at NBA.com;
    # will use 2PT mid-range and in-the-paint made FG for now
    plyr_box_stats_df['FGM_PAINT'] = fgm_paint_list
    plyr_box_stats_df['FGM_2PT_MR'] = fgm_mr_list

    game_dates = plyr_box_stats_df['GAME_DATE'].values.tolist()
    game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') for date in game_dates]
    # add converted python GAME_DATE_OBJ datetime64 object to plyr_box_stats_df
    plyr_box_stats_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj

    # Compile box stats containing stat totals from the past week (7 days before) of game_date for the following: 
    # Total minutes played - pw_min
    # Total games played - pw_games
    # 2-point field goal attempts - pw_fg2a
    # 3-point field goal attempts - pw_gf3a
    # Blocks - pw_blk
    # Rebounds - pw_reb (need offensive/defensive rebounds…?)
    # Personal fouls - pw_pf
    # Assists - pw_ast
    # Other - ???

    pw_mins = []
    pw_gamess = []
    pw_fg3as = []
    pw_fg2as = []
    pw_orebs = []
    pw_drebs = []
    pw_asts = []
    pw_tovs = []
    pw_stls = []
    pw_blks = []
    pw_posss = []
    pw_pfs = []
    pw_pfds = []
    pw_fgm_paints = []
    pw_fgm_2pt_mrs = []
    pw_pie_avg = []
    pw_usg_pct_avg = []

    # iterate through all games in player box stats and calculate totals/avgs for games the week before
    for index, this_game in plyr_box_stats_df.iterrows():
        games_week_before_df = get_games_past_week(this_game, plyr_box_stats_df)
        pw_min = games_week_before_df['MIN'].sum()
        pw_games = games_week_before_df.shape[0]
        pw_fg3a = games_week_before_df['FG3A'].sum()
        pw_fg2a = games_week_before_df['FGA'].sum() - pw_fg3a
        pw_oreb = games_week_before_df['OREB'].sum()
        pw_dreb = games_week_before_df['DREB'].sum()
        pw_ast = games_week_before_df['AST'].sum()
        pw_tov = games_week_before_df['TOV'].sum()
        pw_stl = games_week_before_df['STL'].sum()
        pw_blk = games_week_before_df['BLK'].sum()
        pw_poss = games_week_before_df['POSS'].sum()
        pw_pf = games_week_before_df['PF'].sum()
        pw_pfd = games_week_before_df['PFD'].sum()
        pw_fgm_paint = games_week_before_df['PTS_PAINT'].sum()/2
        pw_fgm_2p = games_week_before_df['FGM'].sum() - games_week_before_df['FG3M'].sum()
        pw_fgm_2pt_mr = pw_fgm_2p - pw_fgm_paint
        pw_pie = games_week_before_df['PIE'].mean()
        pw_usg_pct = games_week_before_df['USG_PCT'].mean()

        pw_mins.append(pw_min)
        pw_gamess.append(pw_games)
        pw_fg3as.append(pw_fg3a)
        pw_fg2as.append(pw_fg2a)
        pw_orebs.append(pw_oreb)
        pw_drebs.append(pw_dreb)
        pw_asts.append(pw_ast)
        pw_tovs.append(pw_tov)
        pw_stls.append(pw_stl)
        pw_blks.append(pw_blk)
        pw_posss.append(pw_poss)
        pw_pfs.append(pw_pf)
        pw_pfds.append(pw_pfd)
        pw_fgm_paints.append(pw_fgm_paint)
        pw_fgm_2pt_mrs.append(pw_fgm_2pt_mr)
        pw_pie_avg.append(pw_pie)
        pw_usg_pct_avg.append(pw_usg_pct)

    plyr_box_stats_df.loc[:,'PW_MIN'] = pw_mins
    plyr_box_stats_df.loc[:,'PW_GAMES'] = pw_gamess
    plyr_box_stats_df.loc[:,'PW_FG3A'] = pw_fg3as
    plyr_box_stats_df.loc[:,'PW_FG2A'] = pw_fg2as
    plyr_box_stats_df.loc[:,'PW_OREB'] = pw_orebs
    plyr_box_stats_df.loc[:,'PW_DREB'] = pw_drebs
    plyr_box_stats_df.loc[:,'PW_ASTS'] = pw_asts
    plyr_box_stats_df.loc[:,'PW_TOV'] = pw_tovs
    plyr_box_stats_df.loc[:,'PW_STL'] = pw_stls
    plyr_box_stats_df.loc[:,'PW_BLK'] = pw_blks
    plyr_box_stats_df.loc[:,'PW_POS'] = pw_posss
    plyr_box_stats_df.loc[:,'PW_PF'] = pw_pfs
    plyr_box_stats_df.loc[:,'PW_PFD'] = pw_pfds
    plyr_box_stats_df.loc[:,'PW_FGM_PAINT'] = pw_fgm_paints
    plyr_box_stats_df.loc[:,'PW_FGM_2PT_MR'] = pw_fgm_2pt_mrs
    plyr_box_stats_df.loc[:,'PW_PIE_AVG'] = pw_pie_avg
    plyr_box_stats_df.loc[:,'PW_USG_PCT_AVG'] = pw_usg_pct_avg

    # fill na with 0 on past week stats
    #plyr_box_stats_df.loc[:,'PW_PFD'] = plyr_box_stats_df['PW_PFD'].fillna(0)
    plyr_box_stats_df.loc[:,'PW_PIE_AVG'] = plyr_box_stats_df['PW_PIE_AVG'].fillna(0)
    plyr_box_stats_df.loc[:,'PW_USG_PCT_AVG'] = plyr_box_stats_df['PW_USG_PCT_AVG'].fillna(0)

    # get player injury data
    # golbal: il_data, mg_data
    player_il = il_data.loc[il_data['PLAYER_ID'] == PlayerID]
    player_mg = mg_data.loc[mg_data['PLAYER_ID'] == PlayerID]

    # get first date game played for season (at end of list becasue last -> first)
    first_game_season = plyr_box_stats_df.iloc[len(plyr_box_stats_df) - 1]
    first_game_season_date = first_game_season['GAME_DATE_OBJ']

    # get missed game history before season
    player_mg_before_season = player_mg.loc[player_mg['Date_OBJ'] < first_game_season_date]
    player_mg_before_season = player_mg_before_season.loc[pd.isnull(player_mg_before_season['Acquired'])]

    # find date of last missed game before season
    last_mg = player_mg_before_season.iloc[len(player_mg_before_season) - 1]
    last_mg_date = last_mg['Date_OBJ']

    # get injured list history before season
    player_il_before_season = player_il.loc[player_il['Date_OBJ'] < first_game_season_date]
    player_il_before_season = player_il_before_season.loc[pd.isnull(player_il_before_season['Acquired'])]

    # find date of last missed game before season
    last_il = player_il_before_season.iloc[len(player_il_before_season) - 1]
    last_il_date = last_il['Date_OBJ']

    # get last injury or missed date before season
    # will use as inital 'anchor date' to calculate days from last injury
    last_injury_date = max(last_il_date, last_mg_date)

    # get number of career injuries before season
    # will use as inital count to calculate rolling injuries during season
    num_career_injuries = len(player_mg_before_season) + len(player_il_before_season)

    # get list of teams player was on during season (if > 1 -> was traded mid-season)
    player_teams = player_team_dates['TEAM_ABBREVIATION'].unique()

    # injury stats can only support a player on the same team thoughout the season (i.e. not traded mid-season)
    if len(player_teams) == 1:
        plyr_box_stats_df = calc_injury_stats(plyr_box_stats_df, player_teams[0], last_injury_date, num_career_injuries)
    else:
        print('Skipping player ' + plyr_box_stats_df['PLAYER_NAME'][0] + 'injury data who was traded mid-season (to-do)')

    # save game log stats for player to .csv file
    player_name = plyr_box_stats_df['PLAYER_NAME'][0]
    player_name = player_name.replace(" ", "_")
    file_name = player_name + '_' + str(plyr_box_stats_df['PLAYER_ID'][0]) + '_2021_22_nba_reg_game_logs.csv'
    plyr_box_stats_df.to_csv(file_name)
    print('Write to file: ' + file_name)





In [ ]:
# use these headers for every request to NBA.com
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [6]:
Season='2021-22'
PerMode='Totals'

# make a list of NBA measure types to loop through for a player
MeasureTypes = [
    'Base',
    'Advanced',
    'Misc',
    'Scoring',
    'Usage'
]

# make a list of NBA seasons to loop through later
season_list = [
    '1996-97',
    '1997-98',
    '1998-99',
    '1999-00',
    '2000-01',
    '2001-02',
    '2002-03',
    '2003-04',
    '2004-05',
    '2005-06',
    '2006-07',
    '2007-08',
    '2008-09',
    '2009-10',
    '2010-11',
    '2011-12',
    '2012-13',
    '2013-14',
    '2014-15',
    '2015-16',
    '2016-17',
    '2017-18',
    '2018-19',
    '2019-20',
    '2020-21',
    '2021-22',
    '2022-23',
]

team_abbrev = ['BOS', 'GSW', 'DET', 'WAS', 'ATL', 'NOP', 'CHI', 'TOR', 'MEM',
       'MIN', 'CHA', 'UTA', 'PHX', 'POR', 'MIL', 'LAC', 'SAS', 'BKN',
       'NYK', 'DEN', 'IND', 'MIA', 'CLE', 'DAL', 'PHI', 'HOU', 'OKC',
       'ORL', 'SAC', 'LAL']

In [105]:
# get general (Measure=Base) player total stats (PerMode=Totals) for 2021-22 season (Season=2021-22) - example URL request
# get PLAYER_ID, PLAYER_NAME, GP, MIN
players_stats_url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode='+PerMode+'&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+Season+'&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight='
players_stats_response = requests.get(url=players_stats_url, headers=headers).json()
players_stats_headers = players_stats_response['resultSets'][0]['headers']
players_stats_data = players_stats_response['resultSets'][0]['rowSet']
players_stats_df = pd.DataFrame(data=players_stats_data, columns=players_stats_headers)

In [106]:
# get subset of players_stats_df containing only PLAYER_ID, PLAYER_NAME, MIN
player_ids_2021_22 = players_stats_df[['PLAYER_ID', 'PLAYER_NAME','GP','MIN']]

# using stat Totals from NBA.com calculate MPG
mpg_list = []
for x in range(0, player_ids_2021_22.shape[0]):
    mpg = player_ids_2021_22.iloc[x]['MIN']/player_ids_2021_22.iloc[x]['GP']
    mpg_list.append(round(mpg,1))

player_ids_2021_22.loc[:,'MPG'] = mpg_list
player_ids_2021_22

C:\Users\Reed\AppData\Local\Temp\ipykernel_29836\2520852832.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_ids_2021_22.loc[:,'MPG'] = mpg_list


,PLAYER_ID,PLAYER_NAME,GP,MIN,MPG
0,203932,Aaron Gordon,75,2375.418333,31.7
1,1630565,Aaron Henry,6,16.983333,2.8
2,1628988,Aaron Holiday,63,1020.750000,16.2
3,1630174,Aaron Nesmith,52,573.878333,11.0
4,1630598,Aaron Wiggins,50,1208.750000,24.2
...,...,...,...,...,...
600,203897,Zach LaVine,67,2328.078333,34.7
601,1630285,Zavier Simpson,4,174.083333,43.5
602,1630192,Zeke Nnaji,41,697.608333,17.0
603,1630533,Ziaire Williams,62,1346.466667,21.7


In [107]:
# limit study to only players with 30MPG in the 2021-22 NBA Regular Season
high_mpg_players_2021_22_df = player_ids_2021_22.loc[(player_ids_2021_22['MIN'] >= 500) & (player_ids_2021_22['MPG'] >= 30)]
# save this data to a file for reference
high_mpg_players_2021_22_df.to_csv('players_mpg_2021_22_30mpg.csv', index=False)
high_mpg_players_2021_22_df

,PLAYER_ID,PLAYER_NAME,GP,MIN,MPG
0,203932,Aaron Gordon,75,2375.418333,31.7
21,203952,Andrew Wiggins,73,2329.513333,31.9
23,203076,Anthony Davis,40,1404.248333,35.1
24,1630162,Anthony Edwards,72,2465.993333,34.2
34,1628389,Bam Adebayo,56,1825.261667,32.6
...,...,...,...,...,...
571,1629639,Tyler Herro,66,2151.011667,32.6
574,1630169,Tyrese Haliburton,77,2694.131667,35.0
575,1630178,Tyrese Maxey,75,2650.356667,35.3
591,203115,Will Barton,71,2276.463333,32.1


In [102]:
# get 2021-22 NBA Season schedule based on nbastatR::seasons_schedule()
schedule_df = pd.read_csv("schedule_2021_22.csv")

# convert dateGame string into datetime64 object, add it to schedule dataframe
sch_game_dates = schedule_df['dateGame'].values.tolist()
sch_game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in sch_game_dates]
schedule_df.loc[:,'dateGame_OBJ']  = sch_game_dates_obj

# create a dictionary of team to list of games played 
team_dates_dict = {}
for team in team_abbrev:
    team_games = schedule_df.loc[(schedule_df['slugTeamWinner'] == team) | (schedule_df['slugTeamLoser'] == team)]
    team_dates_dict[team]=team_games


In [ ]:
# load player dictionary based on data from nbastatR::players_careers()
player_dict = pd.read_csv('df_nba_player_dict.csv')

In [ ]:
# load all injured list data since 2010 from Pro Sports Transactions
il_data = pd.read_csv('prosportstransactions_scrape_inactivelist_2010_2023.csv')
# convert Date string into datetime64 object, add it to il dataframe
il_dates = il_data['Date'].values.tolist()
il_dates_obj = [datetime.datetime.strptime(date, '%m/%d/%Y') for date in il_dates]
il_data.loc[:,'Date_OBJ']  = il_dates_obj

# if possible map NBA Player IDs to entires in injured list
il_id = []

num_cannot_find = 0

for index, row in il_data.iterrows():
    namePlayer = row['Acquired']
    if pd.isnull(row['Acquired']):
        namePlayer = row['Relinquished']
    player = player_dict.loc[player_dict['namePlayer'] == namePlayer]
    if (player.empty):
        il_id.append(np.nan)
        print('cannot find: ' + str(namePlayer))
        num_cannot_find = num_cannot_find + 1
    else:
        il_id.append(int(round(player['idPlayer'].values[0],0)))

print('cannot find: ' + str(num_cannot_find))

il_data.loc[:,'PLAYER_ID'] = il_id

In [ ]:
# load all missed games data since 2010 from Pro Sports Transactions
mg_data = pd.read_csv('prosportstransactions_scrape_missedgames_2010_2023.csv')
# convert Date string into datetime64 object, add it to mg dataframe
mg_dates = mg_data['Date'].values.tolist()
mg_dates_obj = [datetime.datetime.strptime(date, '%m/%d/%Y') for date in mg_dates]
mg_data.loc[:,'Date_OBJ']  = mg_dates_obj

# if possible map NBA Player IDs to entires in missed games list

mg_id = []

num_cannot_find = 0

for index, row in mg_data.iterrows():
    namePlayer = row['Acquired']
    if pd.isnull(row['Acquired']):
        namePlayer = row['Relinquished']
    player = player_dict.loc[player_dict['namePlayer'] == namePlayer]
    if (player.empty):
        mg_id.append(np.nan)
        print('cannot find: ' + str(namePlayer))
        num_cannot_find = num_cannot_find + 1
    else:
        mg_id.append(round(player['idPlayer'].values[0],0))
        
print('cannot find: ' + str(num_cannot_find))

mg_data.loc[:,'PLAYER_ID'] = mg_id

In [14]:
# example NBA.com 2021-21 (Season=2021-22) players bio (https://stats.nba.com/stats/leaguedashplayerbiostats):
# use for AGE, PLAYER_HEIGHT_INCHES, PLAYER_WEIGHT
plyrs_bio_url = 'https://stats.nba.com/stats/leaguedashplayerbiostats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&Season='+Season+'&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
plyrs_bio_response = requests.get(url=plyrs_bio_url, headers=headers).json()
plyrs_bio_headers = plyrs_bio_response['resultSets'][0]['headers']
plyrs_bio_data = plyrs_bio_response['resultSets'][0]['rowSet']
plyrs_bio_df = pd.DataFrame(data=plyrs_bio_data, columns=plyrs_bio_headers)

['PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'AGE',
 'PLAYER_HEIGHT',
 'PLAYER_HEIGHT_INCHES',
 'PLAYER_WEIGHT',
 'COLLEGE',
 'COUNTRY',
 'DRAFT_YEAR',
 'DRAFT_ROUND',
 'DRAFT_NUMBER',
 'GP',
 'PTS',
 'REB',
 'AST',
 'NET_RATING',
 'OREB_PCT',
 'DREB_PCT',
 'USG_PCT',
 'TS_PCT',
 'AST_PCT']

In [15]:
# example NBA.com 2021-21 (Season=2021-22) player index (https://stats.nba.com/stats/playergamelogs/playerindex):
# do we need anything...? Just get from bio?
plyrs_idx_url = 'https://stats.nba.com/stats/playerindex?College=&Country=&DraftPick=&DraftRound=&DraftYear=&Height=&Historical=1&LeagueID=00&Season='+Season+'&SeasonType=Regular%20Season&TeamID=0&Weight='
plyrs_idx_response = requests.get(url=plyrs_idx_url, headers=headers).json()
plyrs_idx_headers = plyrs_idx_response['resultSets'][0]['headers']
plyrs_idx_data = plyrs_idx_response['resultSets'][0]['rowSet']
plyrs_idx_df = pd.DataFrame(data=plyrs_idx_data, columns=plyrs_idx_headers)

['PERSON_ID',
 'PLAYER_LAST_NAME',
 'PLAYER_FIRST_NAME',
 'PLAYER_SLUG',
 'TEAM_ID',
 'TEAM_SLUG',
 'IS_DEFUNCT',
 'TEAM_CITY',
 'TEAM_NAME',
 'TEAM_ABBREVIATION',
 'JERSEY_NUMBER',
 'POSITION',
 'HEIGHT',
 'WEIGHT',
 'COLLEGE',
 'COUNTRY',
 'DRAFT_YEAR',
 'DRAFT_ROUND',
 'DRAFT_NUMBER',
 'ROSTER_STATUS',
 'PTS',
 'REB',
 'AST',
 'STATS_TIMEFRAME',
 'FROM_YEAR',
 'TO_YEAR']

In [ ]:
# selection subset of attributes from each game log measure

plyr_base_headers_subset = ['SEASON_YEAR',
 'PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG3M',
 'FG3A',
 'FTM',
 'FTA',
 'OREB',
 'DREB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'PF',
 'PFD',
 'PTS']

plyr_advanced_headers_subset = ['USG_PCT',
 'PIE',
 'POSS']

plyr_misc_headers_subset= [ 'PTS_PAINT']

plyr_scor_headers_subset = ['PCT_PTS_2PT_MR']

In [ ]:
# PlayerID = 2544 # leBron James
# PlayerID = 202695 # kawhi leoard - lots of time off
# PlayerID = 201935 # James Harden - had 2021-22 trade BRK -> PHI
PlayerID = 1629029 # Luka Doncic

In [ ]:
# loop through PLAYER_ID in high_mpg_players_2021_22_df to generate .csv game logs
PlayerIDs = high_mpg_players_2021_22_df['PLAYER_ID'].values.tolist()

In [18]:
# get player game logs for PlayerID from NBA.com for all measure types (MeasureType=Base,Advanced,Misc,Scoring,Usage)
# PlayerID = 1629029 # Luka Doncic
# Season='2021-22'
# PerMode='Totals'
plyr_base_df = get_game_logs_df(PlayerID,'Base',Season, PerMode)
plyr_advanced_df = get_game_logs_df(PlayerID,'Advanced',Season, PerMode)
plyr_misc_df = get_game_logs_df(PlayerID,'Misc',Season, PerMode)
plyr_scor_df = get_game_logs_df(PlayerID,'Scoring',Season, PerMode)
plyr_usage_df = get_game_logs_df(PlayerID,'Usage',Season, PerMode)

In [19]:
# get subset of attributes from each game log measure
plyr_base_df_subset = plyr_base_df[plyr_base_headers_subset]
plyr_advanced_df_subset = plyr_advanced_df[plyr_advanced_headers_subset]
plyr_misc_df_subset = plyr_misc_df[plyr_misc_headers_subset]
plyr_scor_df_subset = plyr_scor_df[plyr_scor_headers_subset]

# put all game log subsets for player into one dataframe
plyr_box_stats_df = pd.concat([plyr_base_df_subset, 
                               plyr_advanced_df_subset, 
                               plyr_misc_df_subset, 
                               plyr_scor_df_subset], axis=1)

num_box_stats = plyr_box_stats_df.shape[0]

In [20]:
# Add Bio data to game log dataframe: Age, Height (in.), Weight (lbs.)
# global: plyrs_bio_df
player_bio = plyrs_bio_df.loc[plyrs_bio_df['PLAYER_ID'] == PlayerID]

# cannot find player birthday info on NBA.com
# subtract age by one when going back single NBA season; this is the work-around for now
# As of 3/2/2023: for 2021-22 season, substract 1 from age at NBA.com
player_age = [int(player_bio.iloc[0]['AGE'])-1] * num_box_stats
plyr_box_stats_df['AGE'] = player_age

player_height = [player_bio.iloc[0]['PLAYER_HEIGHT_INCHES']] * num_box_stats
plyr_box_stats_df['PLAYER_HEIGHT_INCHES'] = player_height

player_weight = [player_bio.iloc[0]['PLAYER_WEIGHT']] * num_box_stats
plyr_box_stats_df['PLAYER_WEIGHT'] = player_weight

In [21]:
# calculate field goals made mid-range and in the paint
fg_attr = ['FGM','FGA','FG3M','FG3A','FTM','FTA','PTS','PTS_PAINT','PCT_PTS_2PT_MR']

fgm_paint_list = []
fgm_mr_list = []
for x in range(0, num_box_stats):
    game_stats = plyr_box_stats_df.iloc[x][fg_attr]
    fgm_paint = game_stats['PTS_PAINT']/2
    fgm_2p = game_stats['FGM'] - game_stats['FG3M']
    fgm_mr = fgm_2p - fgm_paint
    fgm_paint_list.append(int(fgm_paint))
    fgm_mr_list.append(int(fgm_mr))

# note: cannot find 2PT Mid-range FG attempts in player box scores at NBA.com;
# will use 2PT mid-range and in-the-paint made FG for now
plyr_box_stats_df['FGM_PAINT'] = fgm_paint_list
plyr_box_stats_df['FGM_2PT_MR'] = fgm_mr_list


In [23]:
game_dates = plyr_box_stats_df['GAME_DATE'].values.tolist()
game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') for date in game_dates]
# add converted python GAME_DATE_OBJ datetime64 object to plyr_box_stats_df
plyr_box_stats_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj

In [31]:
'''
Compile box stats containing stat totals from the past week (7 days before) of game_date for the following: 
Total minutes played - pw_min
Total games played - pw_games
2-point field goal attempts - pw_fg2a
3-point field goal attempts - pw_gf3a
Blocks - pw_blk
Rebounds - pw_reb (need offensive/defensive rebounds…?)
Personal fouls - pw_pf
Assists - pw_ast
'''

'\nCompile box stats containing stat totals from the past week (7 days before) of game_date for the following: \nTotal minutes played - pw_min\nTotal games played - pw_games\n2-point field goal attempts - pw_fg2a\n3-point field goal attempts - pw_gf3a\nBlocks - pw_blk\nRebounds - pw_reb (need offensive/defensive rebounds…?)\nPersonal fouls - pw_pf\nAssists - pw_ast\n'

In [44]:
pw_mins = []
pw_gamess = []
pw_fg3as = []
pw_fg2as = []
pw_orebs = []
pw_drebs = []
pw_asts = []
pw_tovs = []
pw_stls = []
pw_blks = []
pw_posss = []
pw_pfs = []
pw_pfds = []
pw_fgm_paints = []
pw_fgm_2pt_mrs = []
pw_pie_avg = []
pw_usg_pct_avg = []

# iterate through all games in player box stats and calculate totals/avgs for games the week before
for index, this_game in plyr_box_stats_df.iterrows():
    games_week_before_df = get_games_past_week(this_game, plyr_box_stats_df)
    pw_min = games_week_before_df['MIN'].sum()
    pw_games = games_week_before_df.shape[0]
    pw_fg3a = games_week_before_df['FG3A'].sum()
    pw_fg2a = games_week_before_df['FGA'].sum() - pw_fg3a
    pw_oreb = games_week_before_df['OREB'].sum()
    pw_dreb = games_week_before_df['DREB'].sum()
    pw_ast = games_week_before_df['AST'].sum()
    pw_tov = games_week_before_df['TOV'].sum()
    pw_stl = games_week_before_df['STL'].sum()
    pw_blk = games_week_before_df['BLK'].sum()
    pw_poss = games_week_before_df['POSS'].sum()
    pw_pf = games_week_before_df['PF'].sum()
    pw_pfd = games_week_before_df['PFD'].sum()
    pw_fgm_paint = games_week_before_df['PTS_PAINT'].sum()/2
    pw_fgm_2p = games_week_before_df['FGM'].sum() - games_week_before_df['FG3M'].sum()
    pw_fgm_2pt_mr = pw_fgm_2p - pw_fgm_paint
    pw_pie = games_week_before_df['PIE'].mean()
    pw_usg_pct = games_week_before_df['USG_PCT'].mean()

    pw_mins.append(pw_min)
    pw_gamess.append(pw_games)
    pw_fg3as.append(pw_fg3a)
    pw_fg2as.append(pw_fg2a)
    pw_orebs.append(pw_oreb)
    pw_drebs.append(pw_dreb)
    pw_asts.append(pw_ast)
    pw_tovs.append(pw_tov)
    pw_stls.append(pw_stl)
    pw_blks.append(pw_blk)
    pw_posss.append(pw_poss)
    pw_pfs.append(pw_pf)
    pw_pfds.append(pw_pfd)
    pw_fgm_paints.append(pw_fgm_paint)
    pw_fgm_2pt_mrs.append(pw_fgm_2pt_mr)
    pw_pie_avg.append(pw_pie)
    pw_usg_pct_avg.append(pw_usg_pct)

plyr_box_stats_df.loc[:,'PW_MIN'] = pw_mins
plyr_box_stats_df.loc[:,'PW_GAMES'] = pw_gamess
plyr_box_stats_df.loc[:,'PW_FG3A'] = pw_fg3as
plyr_box_stats_df.loc[:,'PW_FG2A'] = pw_fg2as
plyr_box_stats_df.loc[:,'PW_OREB'] = pw_orebs
plyr_box_stats_df.loc[:,'PW_DREB'] = pw_drebs
plyr_box_stats_df.loc[:,'PW_ASTS'] = pw_asts
plyr_box_stats_df.loc[:,'PW_TOV'] = pw_tovs
plyr_box_stats_df.loc[:,'PW_STL'] = pw_stls
plyr_box_stats_df.loc[:,'PW_BLK'] = pw_blks
plyr_box_stats_df.loc[:,'PW_POS'] = pw_posss
plyr_box_stats_df.loc[:,'PW_PF'] = pw_pfs
plyr_box_stats_df.loc[:,'PW_PFD'] = pw_pfds
plyr_box_stats_df.loc[:,'PW_FGM_PAINT'] = pw_fgm_paints
plyr_box_stats_df.loc[:,'PW_FGM_2PT_MR'] = pw_fgm_2pt_mrs
plyr_box_stats_df.loc[:,'PW_PIE_AVG'] = pw_pie_avg
plyr_box_stats_df.loc[:,'PW_USG_PCT_AVG'] = pw_usg_pct_avg

# fill na with 0 on past week stats
#plyr_box_stats_df.loc[:,'PW_PFD'] = plyr_box_stats_df['PW_PFD'].fillna(0)
plyr_box_stats_df.loc[:,'PW_PIE_AVG'] = plyr_box_stats_df['PW_PIE_AVG'].fillna(0)
plyr_box_stats_df.loc[:,'PW_USG_PCT_AVG'] = plyr_box_stats_df['PW_USG_PCT_AVG'].fillna(0)

In [49]:
# plyr_box_stats_df.to_csv('Luka_Doncic_1629029_2021_22_game_logs.csv')


In [50]:
# to-do: work on injury stats

In [63]:
player_il = il_data.loc[il_data['PLAYER_ID'] == PlayerID]
player_mg = mg_data.loc[mg_data['PLAYER_ID'] == PlayerID]

In [67]:
# get first date game played for season (at end of list becasue last -> first)
first_game_season = plyr_box_stats_df.iloc[len(plyr_box_stats_df) - 1]
first_game_season_date = first_game_season['GAME_DATE_OBJ']

Timestamp('2021-10-21 00:00:00')

In [68]:
# get missed game history before season
player_mg_before_season = player_mg.loc[player_mg['Date_OBJ'] < first_game_season_date]
player_mg_before_season = player_mg_before_season.loc[pd.isnull(player_mg_before_season['Acquired'])]

,Date,Team,Acquired,Relinquished,Notes,Date_OBJ,PLAYER_ID
11133,1/31/2019,Mavericks,NaN,Luka Doncic,sore left ankle (DTD),2019-01-31,1629029.0
11409,3/30/2019,Mavericks,NaN,Luka Doncic,bruised right thigh (DTD),2019-03-30,1629029.0
11492,4/10/2019,Mavericks,NaN,Luka Doncic,left thigh injury (out for season),2019-04-10,1629029.0
11831,12/15/2019,Mavericks,NaN,Luka Doncic,sprained right ankle (DTD),2019-12-15,1629029.0
12266,8/9/2020,Mavericks,NaN,Luka Doncic,right ankle injury (DTD),2020-08-09,1629029.0
12405,1/3/2021,Mavericks,NaN,Luka Doncic,bruised left quadricep (DTD),2021-01-03,1629029.0
12799,3/11/2021,Mavericks,NaN,Luka Doncic,right ankle injury (DTD),2021-03-11,1629029.0


In [69]:
# find date of last missed game before season
last_mg = player_mg_before_season.iloc[len(player_mg_before_season) - 1]
last_mg_date = last_mg['Date_OBJ']

Timestamp('2021-03-11 00:00:00')

In [70]:
# get injured list history before season
player_il_before_season = player_il.loc[player_il['Date_OBJ'] < first_game_season_date]
player_il_before_season = player_il_before_season.loc[pd.isnull(player_il_before_season['Acquired'])]

,Date,Team,Acquired,Relinquished,Notes,Date_OBJ,PLAYER_ID
12875,12/2/2018,Mavericks,NaN,Luka Doncic,placed on IL with strained right hip,2018-12-02,1629029.0
13607,2/22/2019,Mavericks,NaN,Luka Doncic,placed on IL with sore right ankle,2019-02-22,1629029.0
13813,3/16/2019,Mavericks,NaN,Luka Doncic,placed on IL with bruised right knee,2019-03-16,1629029.0
13955,3/31/2019,Mavericks,NaN,Luka Doncic,placed on IL with bruised right thigh,2019-03-31,1629029.0
14004,4/5/2019,Mavericks,NaN,Luka Doncic,placed on IL with bruised right thigh,2019-04-05,1629029.0
14631,12/16/2019,Mavericks,NaN,Luka Doncic,placed on IL with sprained right ankle,2019-12-16,1629029.0
15104,1/31/2020,Mavericks,NaN,Luka Doncic,placed on IL with sprained right ankle,2020-01-31,1629029.0
15289,2/22/2020,Mavericks,NaN,Luka Doncic,placed on IL with sore right ankle,2020-02-22,1629029.0
15356,3/1/2020,Mavericks,NaN,Luka Doncic,placed on IL with sprained left thumb,2020-03-01,1629029.0
16461,3/3/2021,Mavericks,NaN,Luka Doncic,placed on IL with tightness in lower back,2021-03-03,1629029.0


In [72]:
# find date of last missed game before season
last_il = player_il_before_season.iloc[len(player_il_before_season) - 1]
last_il_date = last_il['Date_OBJ']

Timestamp('2021-04-29 00:00:00')

In [73]:
# get last injury or missed date before season
# will use as inital 'anchor date' to calculate days from last injury
last_injury_date = max(last_il_date, last_mg_date)

Timestamp('2021-04-29 00:00:00')

In [74]:
# get number of career injuries before season
# will use as inital count to calculate rolling injuries during season
num_career_injuries = len(player_mg_before_season) + len(player_il_before_season)

20

In [85]:
# get list of teams player was on during season (if > 1 -> was traded mid-season)
player_teams = player_team_dates['TEAM_ABBREVIATION'].unique()

array(['DAL'], dtype=object)

In [98]:
# injury stats can only support a player on the same team thoughout the season (i.e. not traded mid-season)
if len(player_teams) == 1:
    plyr_box_stats_df = calc_injury_stats(plyr_box_stats_df, player_teams[0], last_injury_date, num_career_injuries)
else:
    print('Skipping player ' + plyr_box_stats_df['PLAYER_NAME'][0] + 'who was traded mid-season (to-do)')
    

C:\Users\Reed\AppData\Local\Temp\ipykernel_29836\2289958321.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_injury_data.loc[:,'CAREER_INJURIES'] = career_injuries
C:\Users\Reed\AppData\Local\Temp\ipykernel_29836\2289958321.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_injury_data.loc[:,'DAYS_LAST_INJURY'] = days_last_injury


In [99]:
plyr_box_stats_df

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,PW_POS,PW_PF,PW_PFD,PW_FGM_PAINT,PW_FGM_2PT_MR,PW_PIE_AVG,PW_USG_PCT_AVG,OUT_NEXT_GAME,CAREER_INJURIES,DAYS_LAST_INJURY
0,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101219,2022-04-10T00:00:00,DAL vs. SAS,W,...,210,6,24,15.0,2.0,0.216667,0.431333,0,26,20
1,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101209,2022-04-08T00:00:00,DAL vs. POR,W,...,222,9,23,14.0,5.0,0.160333,0.417000,0,26,18
2,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101190,2022-04-06T00:00:00,DAL @ DET,W,...,229,6,19,19.0,5.0,0.192000,0.390333,0,26,16
3,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101167,2022-04-03T00:00:00,DAL @ MIL,W,...,270,8,25,27.0,6.0,0.220000,0.390000,0,26,13
4,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022101152,2022-04-01T00:00:00,DAL @ WAS,L,...,270,5,24,24.0,5.0,0.229750,0.358500,0,26,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100075,2021-10-29T00:00:00,DAL @ DEN,L,...,220,9,16,20.0,6.0,0.168667,0.356000,0,20,183
61,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100069,2021-10-28T00:00:00,DAL vs. SAS,W,...,228,7,19,14.0,8.0,0.179667,0.329667,0,20,182
62,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100052,2021-10-26T00:00:00,DAL vs. HOU,W,...,153,4,12,9.0,4.0,0.154500,0.318000,0,20,180
63,2021-22,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022100029,2021-10-23T00:00:00,DAL @ TOR,W,...,77,1,6,2.0,2.0,0.136000,0.276000,0,20,177


In [ ]:
# save game log stats for player
player_name = plyr_box_stats_df['PLAYER_NAME'][0]
player_name = player_name.replace(" ", "_")
file_name = player_name + '_' + str(plyr_box_stats_df['PLAYER_ID'][0]) + '_2021_22_nba_reg_game_logs.csv'
plyr_box_stats_df.to_csv(file_name)